<a href="https://colab.research.google.com/github/JoYongJIn/YongJin-Repository/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pytorch를 이용해 간단한 모델 만들어보기

In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [19]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
training_data

In [ ]:
batch_size = 64 # GPU(하드웨어가속기)를 사용하여 병렬 처리를 최적화하기 위한 배치 사이즈를 적절히 설정해 볼수있다.

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
X[0]

In [ ]:
X[0].shape

In [ ]:
import numpy as np
np.unique(y.numpy())

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(X[1].squeeze().numpy())
plt.title(y[1])

In [ ]:
# 모델 만들기

In [ ]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda" # cuda란 엔비디아에서 개발한 GPU를 활용한 병렬 프로그래밍 모델이다
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module): # nn모듈을 상속받는 neuralnetwork 모델을 정의하여 클래스를 만들었다
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # Flatten은 다차원 텐서를 2차원 텐서로 변환하는 기능
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10) # 10개의 lable중에 정답을 맞추는 분류문제이기 때문에 10이다.
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

In [ ]:
# 모델 매개변수 최적화하기

In [26]:
loss_fn = nn.CrossEntropyLoss() # 손실함수를 crossentrypy로 하고 옵티마이저를 SGD로 한다.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # stochastic gradent descent(확률적 경사 하강법)

In [27]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader): # 배치단위로 학습 데이터셋에 대한 예측수행
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # 예측 오류를 역전파하여 모델의 매개변수를 조정

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [28]:
 # 모델이 학습을 잘 하고있는지 확인하기 위해 테스트 데이터셋으로 모델의 성능을 확인
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n") # 정확도 계산하여 출력

In [ ]:
# 학습을 시키고 epochs를 거듭함에 따라 정확도가 좋아짐을 볼수있다.
epochs = 5 # overfitting이 발생하지 않도록 주의하여 에포크의 크기를 설정하였다.
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
# 모델 저장하기(모델 학습한것을 저장하여 다시 불러와서 쓸수있다. 모델이 학습할때 시간이 걸리는데 그때문에 저장한다.)
# 매개변수등 내부 상태를 저장한다.

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
# 모델 불러오기

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

In [30]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [32]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[1][0], test_data[1][1] # 테스트 데이터를 바꾸어 가며 모델이 잘 예측을 하는지 확인해 볼수있다.
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Pullover"


In [ ]:
# 텐서(tensor)
'''
텐서(tensor)는 배열(array)이나 행렬(matrix)과 매우 유사한 특수한 자료구조입니다.
PyTorch에서는 텐서를 사용하여 모델의 입력(input)과 출력(output), 그리고 모델의 매개변수들을 부호화(encode)합니다.
'''

In [2]:
import torch
import numpy as np

In [ ]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
type(data), type(x_data)

In [5]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
type(np_array), type(x_np) # array가 tensor 형태로 바뀌었다. 파이토치에서는 텐서로 해야 연산된다.

(numpy.ndarray, torch.Tensor)

In [ ]:
x_ones = torch.ones_like(x_data) # x_data의 속성을 유지합니다.
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # x_data의 속성을 덮어씁니다.
print(f"Random Tensor: \n {x_rand} \n")

In [ ]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")
rand_tensor.shape, ones_tensor.shape, zeros_tensor.shape # random, 1, 0으로 tensor 생성

In [ ]:
# 텐서 연산

In [7]:
# GPU가 존재하면 텐서를 이동합니다
if torch.cuda.is_available():
    tensor = tensor.to("cuda") # cuda란 엔비디아에서 개발한 GPU를 활용한 병렬 프로그래밍 모델이다

In [ ]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0 # 인덱싱과 슬라이싱 (사용방법이 넘파이와 매우 유사하다)
print(tensor)

In [ ]:
# 텐서를 NumPy 배열로 변환하기

In [9]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


In [10]:
n = np.ones(5)
t = torch.from_numpy(n) # 넘파이를 텐서로

In [ ]:
# DATASET과 DATALOADER

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


training_data = datasets.FashionMNIST( # "Modified National Institute of Standards and Technology"의 약자로 손으로 쓴 숫자들의 대규모 데이터셋이다.
    root="data", # 여기서는 의류가 레이블이다(원래는 0부터 9까지의 숫자이다)
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show() # 데이터를 순회하고 시각화하기

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, names=['file_name', 'label'])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform #__init__함수는 객체가 생성될 때 자동으로 호출되는 '메서드'이다. 생산자라고 볼수있다. 클래스의 객체가 생성될 때 해당 객체의 초기 상태를 설정한다.

    def __len__(self):
        return len(self.img_labels) # __len__함수는 데이터셋의 샘플 개수를 반환한다.

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1] # __getitem__함수는 주어진 인덱스에 해당하는 샘플을 데이터셋에서 불러오고 반환한다.
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label # 사용자 정의 데이터셋을 만들어 데이터를 수집하거나 데이터를 원하는 형식으로 가공 구조화 하기

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

# 이미지와 정답(label)을 표시합니다.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}") # 학습용 데이터 준비하고 데이터로더로 순회하기

In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
) # 이미지 변환하고 원핫인코딩으로 label 변환하기(학습에 용이하도록(범주형 데이터를 수치화 하여 손실함수와 활성함수와의 상호작용에 용이하도록 한다는것이다.))

In [ ]:
target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1)) # lambda함수를 통한 y(원본 레이블값)를 one-hot-encoding